# Fraud Detection - Data Preprocessing

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Load Raw Data

In [ ]:
df = pd.read_csv("../DATA/raw/financial_fraud_detection_dataset.csv")
print(f"Initial shape: {df.shape}")
df.head()

## Handle Missing Values

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isna().sum())

Missing values per column:
transaction_id                       0
timestamp                            0
sender_account                       0
receiver_account                     0
amount                               0
transaction_type                     0
merchant_category                    0
location                             0
device_used                          0
is_fraud                             0
fraud_type                     4820447
time_since_last_transaction     896513
spending_deviation_score             0
velocity_score                       0
geo_anomaly_score                    0
payment_channel                      0
ip_address                           0
device_hash                          0
dtype: int64


In [ ]:
imputer = SimpleImputer(strategy="mean")
df[df.select_dtypes(include=np.number).columns] = imputer.fit_transform(
    df.select_dtypes(include=np.number)
)

In [ ]:
df.isna().sum()

transaction_id                       0
timestamp                            0
sender_account                       0
receiver_account                     0
amount                               0
transaction_type                     0
merchant_category                    0
location                             0
device_used                          0
is_fraud                             0
fraud_type                     4820447
time_since_last_transaction          0
spending_deviation_score             0
velocity_score                       0
geo_anomaly_score                    0
payment_channel                      0
ip_address                           0
device_hash                          0
dtype: int64

## Drop Unnecessary Columns

In [ ]:
columns_to_drop = ['transaction_id', 'fraud_type']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
print(f"Shape after dropping columns: {df.shape}")

Shape after dropping columns: (5000000, 16)


## Parse Datetime Features

In [ ]:
# Convert timestamp to datetime and extract temporal features
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.day
df['day_of_week'] = df['timestamp'].dt.weekday
df['month'] = df['timestamp'].dt.month

print("Temporal features extracted:")
print(df[['timestamp', 'hour', 'day', 'day_of_week', 'month']].head())

Temporal features extracted:
                   timestamp  hour   day  day_of_week  month
0 2023-08-22 09:22:43.516168   9.0  22.0          1.0    8.0
1 2023-08-04 01:58:02.606711   1.0   4.0          4.0    8.0
2 2023-05-12 11:39:33.742963  11.0  12.0          4.0    5.0
3 2023-10-10 06:04:43.195112   6.0  10.0          1.0   10.0
4 2023-09-24 08:09:02.700162   8.0  24.0          6.0    9.0


In [ ]:
df[df['hour'].isna()]

,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash,hour,day,day_of_week,month
1891874,NaT,ACC375307,ACC630432,195.84,deposit,grocery,Dubai,web,False,4429.861012,0.87,5.0,0.85,wire_transfer,50.135.186.243,D8972916,NaN,NaN,NaN,NaN
2833396,NaT,ACC969484,ACC369350,4.55,payment,travel,Dubai,web,False,-352.835279,1.20,9.0,0.58,wire_transfer,228.70.5.198,D3068177,NaN,NaN,NaN,NaN
3214871,NaT,ACC444490,ACC483161,28.20,transfer,online,Singapore,atm,False,6481.381027,-1.32,12.0,0.32,ACH,234.146.217.89,D1189472,NaN,NaN,NaN,NaN


## Convert Target Variable

In [ ]:
# Convert is_fraud to integer
df['is_fraud'] = df['is_fraud'].astype(int)
print(f"Target variable distribution:\n{df['is_fraud'].value_counts()}")

Target variable distribution:
is_fraud
0    4820447
1     179553
Name: count, dtype: int64


## Sort Data Chronologically

In [ ]:
# Sort data by timestamp - CRITICAL for time-based features
df = df.sort_values('timestamp').reset_index(drop=True)
print(f"Data sorted by timestamp. Shape: {df.shape}")

Data sorted by timestamp. Shape: (5000000, 20)


## Encode Categorical Variables

In [ ]:
# Apply label encoding to categorical columns
print("Encoding categorical variables...")
for col in df.select_dtypes(include=["object","category"]).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    print(f"  - Encoded: {col}")

df.head()

Encoding categorical variables...
  - Encoded: sender_account
  - Encoded: receiver_account
  - Encoded: transaction_type
  - Encoded: merchant_category
  - Encoded: location
  - Encoded: device_used
  - Encoded: payment_channel
  - Encoded: ip_address
  - Encoded: device_hash


,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash,hour,day,day_of_week,month
0,2023-01-01 00:09:26.241974,158871,89037,12.25,2,1,1,0,0,-7559.966086,-0.32,2.0,0.96,1,4781010,3735305,0.0,1.0,6.0,1.0
1,2023-01-01 00:11:36.452582,376957,4992,1347.27,0,6,5,1,0,1.525799,-0.68,5.0,0.67,1,232516,550645,0.0,1.0,6.0,1.0
2,2023-01-01 00:12:48.028557,727819,491398,20.79,1,6,6,3,0,-4568.663237,0.14,8.0,0.42,3,2541431,3740925,0.0,1.0,6.0,1.0
3,2023-01-01 00:21:19.560899,545869,270273,740.73,0,7,0,1,0,1.525799,-2.09,20.0,0.86,1,609531,1437485,0.0,1.0,6.0,1.0
4,2023-01-01 00:23:15.259766,880977,66830,228.67,0,1,1,1,0,-48.738700,0.16,18.0,0.82,1,736628,1085200,0.0,1.0,6.0,1.0


In [ ]:
df.isna().sum()

timestamp                      3
sender_account                 0
receiver_account               0
amount                         0
transaction_type               0
merchant_category              0
location                       0
device_used                    0
is_fraud                       0
time_since_last_transaction    0
spending_deviation_score       0
velocity_score                 0
geo_anomaly_score              0
payment_channel                0
ip_address                     0
device_hash                    0
hour                           3
day                            3
day_of_week                    3
month                          3
dtype: int64

## Data Quality Checks

In [ ]:
# Check for any remaining issues
print("Data Quality Summary:")
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"Missing values: {df.isna().sum().sum()}")
print(f"Duplicate rows: {df.duplicated().sum()}")

Data Quality Summary:
Total rows: 5000000
Total columns: 20
Missing values: 15
Duplicate rows: 0


In [ ]:
df = df.dropna(subset=['timestamp', 'hour', 'day', 'day_of_week', 'month'])

In [ ]:
# Check for any remaining issues
print("Data Quality Summary:")
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"Missing values: {df.isna().sum().sum()}")
print(f"Duplicate rows: {df.duplicated().sum()}")

Data Quality Summary:
Total rows: 4999997
Total columns: 20
Missing values: 0
Duplicate rows: 0


In [ ]:
# Display data types
print("\nData Types:")
print(df.dtypes)


Data Types:
timestamp                      datetime64[us]
sender_account                          int64
receiver_account                        int64
amount                                float64
transaction_type                        int64
merchant_category                       int64
location                                int64
device_used                             int64
is_fraud                                int64
time_since_last_transaction           float64
spending_deviation_score              float64
velocity_score                        float64
geo_anomaly_score                     float64
payment_channel                         int64
ip_address                              int64
device_hash                             int64
hour                                  float64
day                                   float64
day_of_week                           float64
month                                 float64
dtype: object


In [ ]:
# Display basic statistics
df.describe()

,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash,hour,day,day_of_week,month
count,4999997,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06,4.999997e+06
mean,2023-07-03 00:10:10.418651,4.483520e+05,4.483735e+05,3.589344e+02,1.499402e+00,3.500390e+00,3.500437e+00,1.499932e+00,3.591062e-02,1.523688e+00,-3.882662e-04,1.050132e+01,5.000292e-01,1.500378e+00,2.498523e+06,1.917832e+06,1.149665e+01,1.572276e+01,2.999544e+00,6.527235e+00
min,2023-01-01 00:09:26.241974,0.000000e+00,0.000000e+00,1.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.777814e+03,-5.260000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
25%,2023-04-02 17:32:38.758834,2.242590e+05,2.242040e+05,2.657000e+01,0.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,-1.920275e+03,-6.800000e-01,5.000000e+00,2.500000e-01,0.000000e+00,1.249276e+06,9.587620e+05,5.000000e+00,8.000000e+00,1.000000e+00,4.000000e+00
50%,2023-07-03 01:27:16.670582,4.483290e+05,4.484920e+05,1.386700e+02,1.000000e+00,4.000000e+00,4.000000e+00,1.000000e+00,0.000000e+00,1.525799e+00,0.000000e+00,1.100000e+01,5.000000e-01,2.000000e+00,2.498488e+06,1.917991e+06,1.100000e+01,1.600000e+01,3.000000e+00,7.000000e+00
75%,2023-10-02 05:43:06.393417,6.725690e+05,6.724110e+05,5.038900e+02,2.000000e+00,5.000000e+00,6.000000e+00,3.000000e+00,0.000000e+00,1.923793e+03,6.700000e-01,1.600000e+01,7.500000e-01,3.000000e+00,3.747773e+06,2.876538e+06,1.700000e+01,2.300000e+01,5.000000e+00,1.000000e+01
max,2024-01-01 22:58:30.131850,8.965120e+05,8.966380e+05,3.520570e+03,3.000000e+00,7.000000e+00,7.000000e+00,3.000000e+00,1.000000e+00,8.757758e+03,5.020000e+00,2.000000e+01,1.000000e+00,3.000000e+00,4.997067e+06,3.835722e+06,2.300000e+01,3.100000e+01,6.000000e+00,1.200000e+01
std,NaN,2.587920e+05,2.588069e+05,4.699334e+02,1.117896e+00,2.291134e+00,2.291127e+00,1.117982e+00,1.860673e-01,3.240097e+03,1.000807e+00,5.766844e+00,2.886350e-01,1.118295e+00,1.442522e+06,1.107195e+06,6.921329e+00,8.796412e+00,2.003172e+00,3.447722e+00


## Save Processed Data


Preprocessed data saved to: ../DATA/processed.csv
Final shape: (4999997, 20)
